Imports and File Paths

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import json
import scipy.stats as st
from scipy.stats import linregress

#Import API key

from api_keys import weather_api_key


#citipy


from citipy import citipy



# Range of latitudes and longitudes


lat_range = (-90, 90)
lng_range = (-180, 180)

Generate list of cities with criteria

In [ ]:

# List for holding lat amd lngs and cities


lat_lngs = []
cities = []



# Create a set of random lat and lng combinations


lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)



# Identify nearest city for each lat, lng combination and see if unique


for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    if city not in cities:
        cities.append(city)

In [ ]:

city_list = []
cloudiness_list = []
country_list = []
date_list = []
humidity_list = []
lat_list = []
lng_list = []
maxtemp_list = []
wind_list = []
index_counter = 0
set_counter = 1

Perform API Calls and print a log of the cities

In [ ]:

print("Beginning Data Retrieval ")
print("-----------------------------")

base_url = "http://api.openweathermap.org/data/2.5/weather?"
units = "imperial"
query_url = f"{base_url}appid={weather_api_key}&units={units}&q="


# For each city name in cities list


for index, city in enumerate(cities, start = 1):
    try:
        response = requests.get(query_url + city).json()
        city_list.append(response["name"])
        cloudiness_list.append(response["clouds"]["all"])
        country_list.append(response["sys"]["country"])
        date_list.append(response["dt"])
        humidity_list.append(response["main"]["humidity"])
        lat_list.append(response["coord"]["lat"])
        lng_list.append(response["coord"]["lon"])
        maxtemp_list.append(response['main']['temp_max'])
        wind_list.append(response["wind"]["speed"])
        if index_counter > 49:
            index_counter = 0
            set_counter = set_counter + 1
    
        else:
            index_counter = index_counter + 1
            
        print(f"Processing Record {index_counter} of Set {set_counter} : {city}") 
  
    except(KeyError, IndexError):
        print("City not found. Skipping...")

print("-----------------------------")
print("Data Retrieval Complete")
print("-----------------------------")

Convert the data into a dataframe

In [ ]:
weather_df = pd.DataFrame({ 
                "City" : city_list,
                "Country" : country_list,
                "Date" : date_list,
                "Lat" : lat_list,
                "Lng" : lng_list,
                "Cloudiness" : cloudiness_list,
                "Humidity" : humidity_list,
                "Max Temp" : maxtemp_list,
                "Wind Speed" : wind_list
})

In [ ]:
weather_df

Save the data into a csv

In [ ]:
weather_df.to_csv("output_data/cities.csv", index = False)

Plot Data and save images to Graphs folder

In [ ]:
# Create Latitude vs. Cloudiness Scatter

plt.scatter(weather_df["Lat"], weather_df["Cloudiness"], facecolor = "blue", edgecolor = "black")



# Set title

plt.title("City Latitude vs. Cloudiness")


# Set x axis label

plt.xlabel("Latitude")


# Set y axis label

plt.ylabel("Cloudiness (%)")


# Set y axis limit

plt.ylim(-5,105)


# Set grid line

plt.grid(linestyle='-', linewidth=1, alpha = 0.5)


# Save the plotted figure as .pngs

plt.savefig("../Graphs/City Latitude vs Cloudiness")

In [ ]:
# Create Latitude vs. Humidity Scatter

plt.scatter(weather_df["Lat"], weather_df["Humidity"], facecolor = "blue", edgecolor = "black")



# Set title

plt.title("City Latitude vs. Humidity")


# Set x axis label

plt.xlabel("Latitude")


# Set y axis label

plt.ylabel("Humidity (%)")


# Set y axis limit

plt.ylim(-5,105)


# Set grid line

plt.grid(linestyle='-', linewidth=1, alpha = 0.5)


# Save the plotted figure as .pngs

plt.savefig("../Graphs/City Latitude vs Humidity")

In [ ]:
# Create Latitude vs. Max Temperature Scatter

plt.scatter(weather_df["Lat"], weather_df["Max Temp"], facecolor = "blue", edgecolor = "black")



# Set title

plt.title("City Latitude vs. Max Temperature")


# Set x axis label

plt.xlabel("Latitude")


# Set y axis label

plt.ylabel("Max Temperature (F)")


# Set y axis limit

plt.ylim(-5,105)


# Set grid line

plt.grid(linestyle='-', linewidth=1, alpha = 0.5)


# Save the plotted figure as .pngs

plt.savefig("../Graphs/City Latitude vs Max Temperature")

In [ ]:
# Create Latitude vs. Wind Speed Scatter

plt.scatter(weather_df["Lat"], weather_df["Wind Speed"], facecolor = "blue", edgecolor = "black")



# Set title

plt.title("City Latitude vs. Wind Speed")


# Set x axis label

plt.xlabel("Latitude")


# Set y axis label

plt.ylabel("Wind Speed (mph)")


# Set y axis limit

plt.ylim(-5,50)


# Set grid line

plt.grid(linestyle='-', linewidth=1, alpha = 0.5)


# Save the plotted figure as .pngs

plt.savefig("../Graphs/City Latitude vs Wind Speed")

In [ ]:
# Create Northern and Southern Hemisphere DataFrames
north_hemi = weather_df.loc[weather_df["Lat"] >= 0]
south_hemi = weather_df.loc[weather_df["Lat"] < 0]

In [ ]:
# Define function for linear regression and scatter plot


def linear_regression(x,y):
    print(f"The r-squared is : {round(st.pearsonr(x, y)[0],2)}")
    (slope, intercept, rvalue, pvalue, stderr) = linregress(x, y)
    regression_values = x * slope + intercept
    line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
    plt.scatter(x, y)
    plt.plot(x, regression_values,"r-")
    return line_eq


# Define a function for annotation


def annotate(line_eq, a, b):
    plt.annotate(line_eq,(a,b),fontsize=15,color="red")

Northern Cloudiness Regression

In [ ]:
#Call Functions

equation = linear_regression(north_hemi["Lat"], north_hemi["Cloudiness"])

annotate(equation, 25, 45)


# title

plt.title("Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression")


# xlabel

plt.xlabel("Latitude")


# ylabel

plt.ylabel("Cloudiness (%)")


# Save image

plt.savefig("../Graphs/Northern Hemisphere Cloudiness Linear Regression.png")

Southern Cloudiness Regression

In [ ]:
#Call Functions

equation = linear_regression(south_hemi["Lat"], south_hemi["Cloudiness"])

annotate(equation, -50, 40)


# title

plt.title("Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression")


# xlabel

plt.xlabel("Latitude")


# ylabel

plt.ylabel("Cloudiness (%)")


# Save image

plt.savefig("../Graphs/Southern Hemisphere Cloudiness Linear Regression.png")

Northern Humidity Regression

In [ ]:
#Call Functions

equation = linear_regression(north_hemi["Lat"], north_hemi["Humidity"])

annotate(equation, 40, 0)


# title

plt.title("Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression")


# xlabel

plt.xlabel("Latitude")


# ylabel

plt.ylabel("Humidity (%)")


# Save image

plt.savefig("../Graphs/Northern Hemisphere Humidity Linear Regression.png")

Southern Humidity Regression

In [ ]:
#Call Functions

equation = linear_regression(south_hemi["Lat"], south_hemi["Humidity"])

annotate(equation, -50, 20)


# title

plt.title("Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression")


# xlabel

plt.xlabel("Latitude")


# ylabel

plt.ylabel("Humidity (%)")


# Save image

plt.savefig("../Graphs/Southern Hemisphere Humidity Linear Regression.png")

Northern Max Temp Regression

In [ ]:
#Call Functions

equation = linear_regression(north_hemi["Lat"], north_hemi["Max Temp"])

annotate(equation, 0, 25)


# title

plt.title("Northern Hemisphere - Max Temp (F) vs. Latitude Linear Regression")


# xlabel

plt.xlabel("Latitude")


# ylabel

plt.ylabel("Max Temp (F)")


# Save image

plt.savefig("../Graphs/Northern Hemisphere Max Temp Linear Regression.png")

Southern Max Temp Regression

In [ ]:
#Call Functions

equation = linear_regression(south_hemi["Lat"], south_hemi["Max Temp"])

annotate(equation, -50, 90)


# title

plt.title("Southern Hemisphere - Max Temp (F) vs. Latitude Linear Regression")


# xlabel

plt.xlabel("Latitude")


# ylabel

plt.ylabel("Max Temp (F)")


# Save image

plt.savefig("../Graphs/Southern Hemisphere Max Temp Linear Regression.png")

Northern Wind Speed Regression

In [ ]:
#Call Functions

equation = linear_regression(north_hemi["Lat"], north_hemi["Wind Speed"])

annotate(equation, 0, 40)


# title

plt.title("Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression")


# xlabel

plt.xlabel("Latitude")


# ylabel

plt.ylabel("Wind Speed (mph)")


# Save image

plt.savefig("../Graphs/Northern Hemisphere Wind Speed Linear Regression.png")

Southern Wind Speed Regression

In [ ]:
#Call Functions

equation = linear_regression(south_hemi["Lat"], south_hemi["Wind Speed"])

annotate(equation, -30, 30)


# title

plt.title("Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression")


# xlabel

plt.xlabel("Latitude")


# ylabel

plt.ylabel("Wind Speed (mph)")


# Save image

plt.savefig("../Graphs/Southern Hemisphere Wind Speed Linear Regression.png")